# Goal

The goal of this notebook is to compare the efficiency, measured in flops per watt, of the different GPU frequencies that the Nano allows.

In [1]:
# Data Preprocessing

import os

path = "./data/Nano/square_all_frequency/"
files = os.listdir(path)

data = []

for file_name in files:
    temp = {
        # Inputs
        "device": "",
        "datatype": "",
        "matrix_size": -1,
        "tensor": None,
        "gpu_frequency": -1,

        # Results
        "power_usage": [],
        "flops": -1,
        
        # Calculated Results
        "avg_power": -1,
        "flops_per_watt": -1
    }
    with open(path+file_name, "r") as f:
        temp['device'], temp['datatype'], temp['matrix_size'], temp['tensor'], temp['gpu_frequency'] = file_name.split(".")[0].split("-")
        temp['matrix_size'] = float(temp['matrix_size'])
        temp['tensor'] = True if temp['tensor'].lower() == "tensor" else False
        temp['gpu_frequency'] = float(temp['gpu_frequency'])

        file_data = f.readlines()

        _, temp['power_usage'] = zip(*[d.strip().split(",") for d in file_data[:-1]])
        temp['power_usage'] = list(map(float, temp['power_usage']))
        temp['avg_power'] = sum(temp['power_usage'])/len(temp['power_usage'])

        temp['flops'] = float(file_data[-1])

        temp['flops_per_watt'] = temp['flops'] / temp['avg_power']
    
    data.append(temp)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
plt.ion()

y_opt = widgets.Dropdown(options=["avg_power", "flops", "flops_per_watt"], value="flops_per_watt", description="y axis")

output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10, 7), sharey=True, sharex=True)

search = {
    "datatype": 'float',
    "gpu_frequency": 0,
    "tensor": True
}

search_x = "matrix_size"
search_y = y_opt.value

def refresh_values():
    ax.grid(axis='y')
    ax.set_title("Flops per Watt by Matrix Size")
    ax.set_xlabel("Columns")
    ax.set_ylabel("Flops")
    ax.set_axisbelow(True)
    for idx, f in enumerate([76800000, 153600000, 230400000, 307200000, 384000000, 460800000, 537600000, 614400000, 691200000, 768000000, 844800000, 921600000]):
        search["gpu_frequency"] = f
        filtered = [d for d in data if search.items() <= d.items()]
        x, y = zip(*sorted([(r[search_x], r[search_y]) for r in filtered], key=lambda d : d[0]))
        ax.plot(x, y, label=f"{f/1e9:.2f} GHz", linestyle='-' if f > 614400000 else '--' if f > 307200000 else '-.')
        ax.set_xticks(np.arange(0, 2049, 128))
        ax.legend(loc="upper left")

def update_y(change):
    global search_y
    search_y = change["new"]
    ax.clear()
    if (change["new"] == "flops"):
        ax.set_yticks(np.arange(0, 3e11, 1e11))
    elif (change["new"] == "flops_per_watt"):
        ax.set_yticks(np.arange(0, 200e9, 50e9))
        pass
    else:
        ax.set_yticks(np.arange(0, 11, 1))
    refresh_values()

y_opt.observe(update_y, names="value")

display(
    widgets.HBox([
        widgets.VBox([y_opt]),
        widgets.VBox([output])
    ])
)

refresh_values()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# TODO: Graph Flops/Watt Relative to Max frequency

import matplotlib.figure as figure

matrix_size = 2048

search = {
    "datatype": "float",
    "matrix_size": matrix_size,
    "tensor": True,
    # "gpu_frequency": -1 // This is the x-axis
}

filtered = [d for d in data if search.items() <= d.items()]
x, y = zip(*sorted([(d['gpu_frequency'], d['flops_per_watt']) for d in filtered], key=lambda d : d[0]))


fig, ax = plt.subplots(figsize=(15, 7))
ax.set_axisbelow(True)
ax.grid(axis='y')
ax.set_xlabel("Frequency (MHz)")
ax.set_ylabel("Flops/Watt")

plt.bar([str(xx) for xx in x], y, bottom=0, tick_label=[int(tx/1e6) for tx in x])



fig, ax = plt.subplots(figsize=(15, 7))
ax.set_axisbelow(True)
ax.grid(axis='y')
ax.set_xlabel("Frequency (MHz)")
ax.set_ylabel("Flops/Watt")

plt.bar([str(xx) for xx in x], [(yy/y[-1]) for yy in y], bottom=0, tick_label=[int(tx/1e6) for tx in x])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 12 artists>